# Imports

In [ ]:
from azure.storage.blob import BlobClient
import pandas as pd
import io
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from azure.storage.blob import BlobServiceClient
from io import StringIO


# Connection

In [ ]:
from azure.storage.blob import BlobServiceClient

connect_str = "DefaultEndpointsProtocol=https;AccountName=mlpw5g48966911887;AccountKey=mrAhppNTLycLK1odEtWiGfubfle5TJfUMwex5QV2+Q8Ggk6kMSvBUXwY+JTILISiJjRqNhEIr/oJ+AStpDybJQ==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [2]:
container_name = "flights"
blob_name = "flights_weather.csv"

blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

data = io.BytesIO(blob_client.download_blob().readall())
df = pd.read_csv(data, low_memory=False)

### Making a category for the rain and removing NaN rows

In [3]:
# Remove rows where any of these columns have null values
df.dropna(subset=['DepTime', 'ArrTime', 'DepDelay', 'ArrDelay', 'rain (mm)Dep', 'rain (mm)Arr'], inplace=True)

# Remove diverted flights
df = df[df['Diverted'] == 0]

# Define rain categories
def categorize_rain(amount):
    if amount == 0:
        return 'No rain'
    elif amount <= 5:
        return 'Light rain'
    elif amount <= 15:
        return 'Moderate rain'
    else:
        return 'Heavy rain'

# Apply the categorization to departure and arrival rain data
df['rain_category_Dep'] = df['rain (mm)Dep'].apply(categorize_rain)
df['rain_category_Arr'] = df['rain (mm)Arr'].apply(categorize_rain)

# Optionally, drop the original rain columns if no longer needed
df.drop(['rain (mm)Dep', 'rain (mm)Arr'], axis=1, inplace=True)

# Departures machine learning

In [4]:
# List of columns to keep for the departure model
departure_columns = [
    'DepDelay', 'DepDel15', 'Origin', 'CRSDepTime', 'TaxiOut',
    'rain_category_Dep', 'wind_speed_10m (km/h)Dep',
    'wind_direction_10m (°)Dep', 'temperature_2m (°C)Dep', 'relative_humidity_2m (%)Dep',
    'cloud_cover (%)Dep', 'weather_code (wmo code)Dep'
]

# Reduce the dataframe to only the departure-related columns
df_departures = df[departure_columns]

In [5]:
display(df_departures)

,DepDelay,DepDel15,Origin,CRSDepTime,TaxiOut,rain_category_Dep,wind_speed_10m (km/h)Dep,wind_direction_10m (°)Dep,temperature_2m (°C)Dep,relative_humidity_2m (%)Dep,cloud_cover (%)Dep,weather_code (wmo code)Dep
0,10.0,0.0,SBN,06:00,20.0,No rain,8.0,333.0,16.6,79.0,24.0,1.0
1,-4.0,0.0,SBN,06:00,14.0,No rain,8.1,148.0,11.6,73.0,0.0,0.0
2,-9.0,0.0,SBN,06:00,15.0,No rain,13.0,222.0,20.4,56.0,9.0,0.0
3,-3.0,0.0,SBN,06:00,14.0,No rain,13.7,207.0,22.5,60.0,1.0,0.0
4,3.0,0.0,CLT,14:30,21.0,No rain,9.5,261.0,29.8,45.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5407542,92.0,1.0,IAH,20:05,13.0,No rain,3.6,225.0,33.6,51.0,25.0,1.0
5407544,44.0,1.0,EWR,14:40,94.0,No rain,7.6,251.0,29.3,73.0,54.0,2.0
5407545,39.0,1.0,DEN,19:05,15.0,No rain,4.5,104.0,32.2,30.0,4.0,0.0
5407546,8.0,0.0,ORD,15:50,19.0,Light rain,17.8,21.0,19.3,88.0,99.0,53.0


### Encodig the airports

In [6]:
# List of provided airport codes
airport_codes = [
    "PLN", "BTV", "CWA", "TWF", "ABE", "MLI", "PHF", "OME", "VLD", "ROA", "PSM", "JAN", "DCA", "MDT", "HOB",
    "SCC", "BNA", "ACK", "SHR", "MKE", "LGA", "RDM", "IDA", "OAJ", "MYR", "PSG", "JAX", "MSN", "JMS", "SHD",
    "AUS", "TOL", "CHA", "AMA", "MGM", "ABY", "SAT", "ISP", "MEM", "LBF", "HPN", "ONT", "HYS", "COS", "BGR",
    "BDL", "PAE", "MFR", "SCK", "STX", "ABR", "FLL", "SAN", "FAI", "CSG", "DAL", "ALO", "RAP", "DHN", "MTJ",
    "ECP", "HHH", "ELP", "LGB", "INL", "BRD", "CGI", "AVL", "DEC", "SPS", "RFD", "HGR", "BOI", "PIH", "LWS",
    "SUX", "BRW", "DSM", "BPT", "WYS", "IAD", "MAF", "HOU", "BFL", "OWB", "TUL", "CLT", "ADQ", "PAH", "CNY",
    "HRL", "DTW", "ICT", "ESC", "JLN", "GPT", "TYR", "LBE", "KTN", "GFK", "FWA", "SLC", "OTH", "LIH", "BLI",
    "JNU", "SMX", "HIB", "EAR", "BQN", "MVY", "FAR", "XNA", "BWI", "ORF", "ACV", "USA", "RIW", "LSE", "RSW",
    "ROW", "IAH", "HTS", "FNT", "ALS", "LAX", "PSC", "SRQ", "EGE", "FSD", "GEG", "MKG", "PIE", "BMI", "GST",
    "TLH", "PWM", "BGM", "ATW", "GRR", "BIS", "XWA", "SJC", "LYH", "BUF", "BZN", "AGS", "SEA", "AVP", "ADK",
    "DFW", "PSE", "STT", "SBN", "DEN", "PUW", "LNK", "LWB", "LRD", "PBI", "TXK", "ABI", "FLG", "OKC", "SMF",
    "SUN", "EKO", "MLB", "MBS", "BRO", "CHS", "LAN", "AZA", "TPA", "RST", "MQT", "JAC", "ELM", "SYR", "LBL",
    "DDC", "RNO", "MEI", "ACT", "EYW", "CYS", "SBA", "PVD", "CRP", "IAG", "CDC", "PIA", "SIT", "PBG", "DAY",
    "CIU", "PUB", "YAK", "SJT", "LFT", "APN", "SAV", "BKG", "ITO", "EWR", "ILG", "SNA", "GRK", "GSO", "MSY",
    "LIT", "GSP", "ACY", "HYA", "PSP", "GTR", "DLG", "DBQ", "SGF", "FSM", "GUC", "IND", "TVC", "MRY", "YKM",
    "LAR", "SJU", "PNS", "SWF", "BET", "CLL", "COD", "OGG", "CRW", "DAB", "GRI", "KOA", "CMX", "TYS", "DVL",
    "RHI", "SDF", "MSO", "GRB", "SBP", "SWO", "GJT", "OTZ", "ANC", "HDN", "MIA", "MOT", "BTR", "TBN", "OMA",
    "DIK", "EVV", "PIT", "EWN", "SGU", "EAU", "MHK", "EUG", "YUM", "LBB", "DRO", "MLU", "DRT", "MCI", "PGD",
    "FCA", "GCC", "GNV", "BLV", "HLN", "FAY", "BUR", "MOB", "FOD", "BQK", "CHO", "ASE", "RIC", "COU", "DLH",
    "MSP", "VEL", "ROC", "BHM", "ORD", "VPS", "LAW", "GTF", "CPR", "STC", "CMH", "CMI", "SHV", "ORH", "MHT",
    "PDX", "SLN", "RDU", "CVG", "BJI", "RKS", "CDV", "BIL", "CLE", "ATL", "LEX", "STL", "HSV", "SCE", "RDD",
    "GCK", "PHX", "WRG", "AZO", "EAT", "BOS", "ERI", "SFO", "BTM", "CAK", "LAS", "PIB", "MDW", "OGS", "PVU",
    "CAE", "AKN", "ALB", "PHL", "LCH", "TEX", "SAF", "ABQ", "AEX", "ALW", "BFF", "BIH", "CID", "CKB", "FAT",
    "GGG", "HNL", "ILM", "IMT", "ITH", "JFK", "JST", "LCK", "MCO", "MCW", "MFE", "OAK", "PRC", "SFB", "SPI",
    "STS", "TRI", "TTN", "TUS", "TWS", "VCT"
]

# Creating an encoding map sorted alphabetically
airport_labels = {code: i for i, code in enumerate(sorted(airport_codes))}

# Adding a new column to the DataFrame with the encoded values
df_departures['Origin_encoded'] = df_departures['Origin'].map(airport_labels)

/tmp/ipykernel_5322/2476772047.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_departures['Origin_encoded'] = df_departures['Origin'].map(airport_labels)


### Encoding the rain 

In [7]:
# Creating an instance of LabelEncoder
encoder = LabelEncoder()

# Fitting the encoder and transforming the 'rain_category_Dep' column
df_departures['rain_category_Dep_encoded'] = encoder.fit_transform(df_departures['rain_category_Dep'])

# Optionally, to see the mapping from categorical labels to integers
label_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(label_mapping)

{'Heavy rain': 0, 'Light rain': 1, 'Moderate rain': 2, 'No rain': 3}


/tmp/ipykernel_5322/2330922055.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_departures['rain_category_Dep_encoded'] = encoder.fit_transform(df_departures['rain_category_Dep'])


In [8]:
# Check the first few entries of the new encoded column
print(df_departures[['rain_category_Dep', 'rain_category_Dep_encoded']])

        rain_category_Dep  rain_category_Dep_encoded
0                 No rain                          3
1                 No rain                          3
2                 No rain                          3
3                 No rain                          3
4                 No rain                          3
...                   ...                        ...
5407542           No rain                          3
5407544           No rain                          3
5407545           No rain                          3
5407546        Light rain                          1
5407547           No rain                          3

[5231485 rows x 2 columns]


### Splitting the dataset

In [9]:
# Define features and target
X = df_departures.drop('DepDelay', axis=1)  # all other columns are features
y = df_departures['DepDelay']  # DepDelay is the target

# Split the dataset into training (75%) and testing (25%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Output the sizes of the train and test sets to verify
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

Training set size: 3923613 samples
Testing set size: 1307872 samples


### Preparing the data

In [10]:
# Convert 'CRSDepTime' from 'HH:MM' format to minutes past midnight
df_departures['CRSDepTime'] = df_departures['CRSDepTime'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

# Confirm the transformation
print(df_departures['CRSDepTime'].head())

0    360
1    360
2    360
3    360
4    870
Name: CRSDepTime, dtype: int64


/tmp/ipykernel_5322/1274299914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_departures['CRSDepTime'] = df_departures['CRSDepTime'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))


### Precision, Recall, F1 Score and Support

# 

In [11]:
# Define features and target variable, ensuring no non-numeric columns are included
X = df_departures.drop(['DepDelay', 'Origin', 'rain_category_Dep'], axis=1)  # Adjust this line if there are other non-numeric columns
y = df_departures['DepDelay']

y = y.apply(
    lambda row:
        'early' if row < 0
        else ('0 min - 1 hour' if row <= 60
        else ('1-3 hours' if row <= 180
        else 'more than 3 hours'
        ))
)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Continue with the setup of the RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Predicting and evaluating
y_pred_class = classifier.predict(X_test)

# Printing classification report
report = classification_report(y_test, y_pred_class, digits=4)
print(report)

                   precision    recall  f1-score   support

   0 min - 1 hour     0.5643    0.5090    0.5352    503915
        1-3 hours     0.4045    0.2271    0.2909     96468
            early     0.7257    0.8407    0.7789    683898
more than 3 hours     0.2427    0.0716    0.1105     23591

         accuracy                         0.6537   1307872
        macro avg     0.4843    0.4121    0.4289   1307872
     weighted avg     0.6311    0.6537    0.6370   1307872



### Saving to csv

In [26]:
connect_str = 'DefaultEndpointsProtocol=https;AccountName=mlpw5g48966911887;AccountKey=mrAhppNTLycLK1odEtWiGfubfle5TJfUMwex5QV2+Q8Ggk6kMSvBUXwY+JTILISiJjRqNhEIr/oJ+AStpDybJQ==;EndpointSuffix=core.windows.net'  
container_name = 'azureml' 
blob_name = 'departuresml.csv'  

# Create a blob service client to interact with the blob storage
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

# Convert the DataFrame to a CSV string
output = StringIO()
df_departures.to_csv(output, index=False)
output.seek(0)

# Create a blob in the specified container and upload the data
blob_client = container_client.get_blob_client(blob_name)
blob_client.upload_blob(output.getvalue(), overwrite=True)

print(f"File {blob_name} successfully saved in {container_name} on Azure Blob Storage.")

File departuresml.csv successfully saved in azureml on Azure Blob Storage.


#  Arrivals machine learning

In [15]:
# List of columns to keep for the arrival model
arrival_columns = [
    'ArrDelay', 'ArrDel15', 'Dest', 'CRSArrTime', 'TaxiIn',
    'rain_category_Arr', 'wind_speed_10m (km/h)Arr',
    'wind_direction_10m (°)Arr', 'temperature_2m (°C)Arr', 'relative_humidity_2m (%)Arr',
    'cloud_cover (%)Arr', 'weather_code (wmo code)Arr'
]

# Reduce the dataframe to only the arrival-related columns
df_arrivals = df[arrival_columns]

In [16]:
display(df_arrivals)

,ArrDelay,ArrDel15,Dest,CRSArrTime,TaxiIn,rain_category_Arr,wind_speed_10m (km/h)Arr,wind_direction_10m (°)Arr,temperature_2m (°C)Arr,relative_humidity_2m (%)Arr,cloud_cover (%)Arr,weather_code (wmo code)Arr
0,12.0,0.0,CLT,07:53,23.0,No rain,11.4,198.0,24.0,75.0,100.0,3.0
1,-17.0,0.0,CLT,07:53,11.0,No rain,14.1,15.0,21.0,89.0,100.0,3.0
2,-15.0,0.0,CLT,07:53,20.0,No rain,4.7,212.0,20.1,98.0,100.0,3.0
3,-14.0,0.0,CLT,07:53,15.0,No rain,3.4,108.0,21.0,99.0,100.0,3.0
4,-8.0,0.0,PIA,15:35,5.0,Light rain,12.2,358.0,18.1,88.0,100.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5407542,70.0,1.0,ATL,23:15,4.0,No rain,8.7,355.0,30.5,58.0,100.0,3.0
5407544,107.0,1.0,IAH,17:23,9.0,No rain,3.8,311.0,32.6,56.0,27.0,1.0
5407545,24.0,1.0,RDM,20:37,3.0,No rain,11.9,305.0,30.1,15.0,0.0,0.0
5407546,17.0,1.0,DEN,17:37,9.0,No rain,9.7,141.0,29.3,39.0,10.0,0.0


### Encoding the airports

In [17]:
# List of provided airport codes
airport_codes = [
    "PLN", "BTV", "CWA", "TWF", "ABE", "MLI", "PHF", "OME", "VLD", "ROA", "PSM", "JAN", "DCA", "MDT", "HOB",
    "SCC", "BNA", "ACK", "SHR", "MKE", "LGA", "RDM", "IDA", "OAJ", "MYR", "PSG", "JAX", "MSN", "JMS", "SHD",
    "AUS", "TOL", "CHA", "AMA", "MGM", "ABY", "SAT", "ISP", "MEM", "LBF", "HPN", "ONT", "HYS", "COS", "BGR",
    "BDL", "PAE", "MFR", "SCK", "STX", "ABR", "FLL", "SAN", "FAI", "CSG", "DAL", "ALO", "RAP", "DHN", "MTJ",
    "ECP", "HHH", "ELP", "LGB", "INL", "BRD", "CGI", "AVL", "DEC", "SPS", "RFD", "HGR", "BOI", "PIH", "LWS",
    "SUX", "BRW", "DSM", "BPT", "WYS", "IAD", "MAF", "HOU", "BFL", "OWB", "TUL", "CLT", "ADQ", "PAH", "CNY",
    "HRL", "DTW", "ICT", "ESC", "JLN", "GPT", "TYR", "LBE", "KTN", "GFK", "FWA", "SLC", "OTH", "LIH", "BLI",
    "JNU", "SMX", "HIB", "EAR", "BQN", "MVY", "FAR", "XNA", "BWI", "ORF", "ACV", "USA", "RIW", "LSE", "RSW",
    "ROW", "IAH", "HTS", "FNT", "ALS", "LAX", "PSC", "SRQ", "EGE", "FSD", "GEG", "MKG", "PIE", "BMI", "GST",
    "TLH", "PWM", "BGM", "ATW", "GRR", "BIS", "XWA", "SJC", "LYH", "BUF", "BZN", "AGS", "SEA", "AVP", "ADK",
    "DFW", "PSE", "STT", "SBN", "DEN", "PUW", "LNK", "LWB", "LRD", "PBI", "TXK", "ABI", "FLG", "OKC", "SMF",
    "SUN", "EKO", "MLB", "MBS", "BRO", "CHS", "LAN", "AZA", "TPA", "RST", "MQT", "JAC", "ELM", "SYR", "LBL",
    "DDC", "RNO", "MEI", "ACT", "EYW", "CYS", "SBA", "PVD", "CRP", "IAG", "CDC", "PIA", "SIT", "PBG", "DAY",
    "CIU", "PUB", "YAK", "SJT", "LFT", "APN", "SAV", "BKG", "ITO", "EWR", "ILG", "SNA", "GRK", "GSO", "MSY",
    "LIT", "GSP", "ACY", "HYA", "PSP", "GTR", "DLG", "DBQ", "SGF", "FSM", "GUC", "IND", "TVC", "MRY", "YKM",
    "LAR", "SJU", "PNS", "SWF", "BET", "CLL", "COD", "OGG", "CRW", "DAB", "GRI", "KOA", "CMX", "TYS", "DVL",
    "RHI", "SDF", "MSO", "GRB", "SBP", "SWO", "GJT", "OTZ", "ANC", "HDN", "MIA", "MOT", "BTR", "TBN", "OMA",
    "DIK", "EVV", "PIT", "EWN", "SGU", "EAU", "MHK", "EUG", "YUM", "LBB", "DRO", "MLU", "DRT", "MCI", "PGD",
    "FCA", "GCC", "GNV", "BLV", "HLN", "FAY", "BUR", "MOB", "FOD", "BQK", "CHO", "ASE", "RIC", "COU", "DLH",
    "MSP", "VEL", "ROC", "BHM", "ORD", "VPS", "LAW", "GTF", "CPR", "STC", "CMH", "CMI", "SHV", "ORH", "MHT",
    "PDX", "SLN", "RDU", "CVG", "BJI", "RKS", "CDV", "BIL", "CLE", "ATL", "LEX", "STL", "HSV", "SCE", "RDD",
    "GCK", "PHX", "WRG", "AZO", "EAT", "BOS", "ERI", "SFO", "BTM", "CAK", "LAS", "PIB", "MDW", "OGS", "PVU",
    "CAE", "AKN", "ALB", "PHL", "LCH", "TEX", "SAF", "ABQ", "AEX", "ALW", "BFF", "BIH", "CID", "CKB", "FAT",
    "GGG", "HNL", "ILM", "IMT", "ITH", "JFK", "JST", "LCK", "MCO", "MCW", "MFE", "OAK", "PRC", "SFB", "SPI",
    "STS", "TRI", "TTN", "TUS", "TWS", "VCT"
]

# Creating an encoding map sorted alphabetically
airport_labels = {code: i for i, code in enumerate(sorted(airport_codes))}

# Adding a new column to the DataFrame with the encoded values
df_arrivals['Dest_encoded'] = df_arrivals['Dest'].map(airport_labels)

/tmp/ipykernel_5322/3012298080.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arrivals['Dest_encoded'] = df_arrivals['Dest'].map(airport_labels)


### Encoding the rain

In [18]:
# Creating an instance of LabelEncoder
encoder = LabelEncoder()

# Fitting the encoder and transforming the 'rain_category_Arr' column
df_arrivals['rain_category_Arr_encoded'] = encoder.fit_transform(df_arrivals['rain_category_Arr'])

# Optionally, to see the mapping from categorical labels to integers
label_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(label_mapping)

{'Heavy rain': 0, 'Light rain': 1, 'Moderate rain': 2, 'No rain': 3}


/tmp/ipykernel_5322/2555258198.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arrivals['rain_category_Arr_encoded'] = encoder.fit_transform(df_arrivals['rain_category_Arr'])


In [19]:
# Check the first few entries of the new encoded column
print(df_arrivals[['rain_category_Arr', 'rain_category_Arr_encoded']])

        rain_category_Arr  rain_category_Arr_encoded
0                 No rain                          3
1                 No rain                          3
2                 No rain                          3
3                 No rain                          3
4              Light rain                          1
...                   ...                        ...
5407542           No rain                          3
5407544           No rain                          3
5407545           No rain                          3
5407546           No rain                          3
5407547        Light rain                          1

[5231485 rows x 2 columns]


### Splitting the dataset

In [20]:
# Define features and target
X = df_arrivals.drop('ArrDelay', axis=1)  # all other columns are features
y = df_arrivals['ArrDelay']  # ArrDelay is the target

# Split the dataset into training (75%) and testing (25%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Output the sizes of the train and test sets to verify
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

Training set size: 3923613 samples
Testing set size: 1307872 samples


### Preparing the data

In [21]:
# Convert 'CRSArrTime' from 'HH:MM' format to minutes past midnight
df_arrivals['CRSArrTime'] = df_arrivals['CRSArrTime'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

# Confirm the transformation
print(df_arrivals['CRSArrTime'].head())

0    473
1    473
2    473
3    473
4    935
Name: CRSArrTime, dtype: int64


/tmp/ipykernel_5322/849323295.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arrivals['CRSArrTime'] = df_arrivals['CRSArrTime'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))


In [ ]:
# Define features and target variable, ensuring no non-numeric columns are included
X = df_arrivals.drop(['ArrDelay', 'Dest', 'rain_category_Arr'], axis=1)  # Adjust this line if there are other non-numeric columns
y = df_arrivals['ArrDelay']

y = y.apply(
    lambda row:
        'early' if row < 0
        else ('0 min - 1 hour' if row <= 60
        else ('1-3 hours' if row <= 180
        else 'more than 3 hours'
        ))
)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Continue with the setup of the RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Predicting and evaluating
y_pred_class = classifier.predict(X_test)

# Printing classification report
report = classification_report(y_test, y_pred_class, digits=4)
print(report)

                   precision    recall  f1-score   support

   0 min - 1 hour     0.5634    0.4994    0.5295    442031
        1-3 hours     0.3856    0.2128    0.2742     96728
            early     0.7780    0.8924    0.8313    745585
more than 3 hours     0.1933    0.0612    0.0930     23528

         accuracy                         0.6943   1307872
        macro avg     0.4800    0.4164    0.4320   1307872
     weighted avg     0.6659    0.6943    0.6748   1307872



### Saving to csv

In [27]:
connect_str = 'DefaultEndpointsProtocol=https;AccountName=mlpw5g48966911887;AccountKey=mrAhppNTLycLK1odEtWiGfubfle5TJfUMwex5QV2+Q8Ggk6kMSvBUXwY+JTILISiJjRqNhEIr/oJ+AStpDybJQ==;EndpointSuffix=core.windows.net'  
container_name = 'azureml' 
blob_name = 'arrivalsml.csv'  

blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

output = StringIO()
df_arrivals.to_csv(output, index=False)
output.seek(0)

blob_client = container_client.get_blob_client(blob_name)
blob_client.upload_blob(output.getvalue(), overwrite=True)

print(f"File {blob_name} successfully saved in {container_name} on Azure Blob Storage.")

File arrivalsml.csv successfully saved in azureml on Azure Blob Storage.
